In [159]:
import sqlite3
import json
from os import getenv
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pytesseract
from PIL import Image
import cv2
import numpy as np
from re import split

In [122]:
import csv

fields = ["titles", "url", "current", "latest"]
filename = "tempdb.csv"

with open(filename, "w", encoding="utf-8", newline="") as file:
    csvwriter = csv.writer(file)
    csvwriter.writerow(fields)
    clumps = list(zip(titles, urls, current, latest))
    csvwriter.writerows(clumps)

In [140]:
con = sqlite3.connect("custom.db")
cur = con.cursor()

In [130]:
data = ('something', json.dumps([]), 1, 2)
cur.execute("INSERT INTO manga VALUES (?, ?, ?, ?)", data)



In [411]:
def func(line):
    line = line.strip('"')
    curr_chap, latest_chap = "", ""

    if line[0] == "C": # it starts with "Chapter"
        line = line.split("Chapter")[1:]
        first = line[0].split("Vol")

        if len(first) == 1:
            curr_chap = "Chapter" + line[0][:-1]
            latest_chap = "Chapter " + line[1].strip()
        elif len(first) == 2:
            curr_chap = "Chapter" + first[0][:-1]
            latest_chap = "Vol" + first[1] + "Chapter " + line[-1].strip()

    else: # it starts with "Vol"
        line = line.split("Vol")[1:]

        if len(line) == 1: # the latest chapter starts with "Chapter"
            line = line[0].rsplit(",Chapter", 1)
            curr_chap = "Vol" + line[0]
            latest_chap = "Chapter " + line[-1].strip()
        elif len(line) == 2: # the latest chapter starts with "Vol"
            curr_chap = "Vol" + line[0][:-1]
            latest_chap = "Vol" + line[1].strip()

        # print(f"{curr_chap} --- {latest_chap}")

    return curr_chap, latest_chap

one = "Vol.5 Chapter 43: Cheers,Vol.15 Chapter 165: Takehi Gouda's Ability\n"
two = "Chapter 3,Chapter 11\n"
three = "Chapter 14,Vol.4 Chapter 18\n"
four = "Vol.4 Chapter 14,Chapter 18: Something else is going on\n"

print(func(four))

('Vol.4 Chapter 14', 'Chapter 18: Something else is going on')


In [412]:
titles = []
urls = []
current = []
latest = []
full = []

with open("tempdb.csv", "r", encoding="utf-8") as file:
    csvwriter = csv.reader(file)
    for i, lines in enumerate(file):
        full.append(lines)
        if i == 0:
            continue

        lines = split("https", lines)
        titles.append(lines[0][:-1].strip('"'))

        lines = lines[1].split(",", maxsplit=1)
        urls.append(f"https{lines[0]}")
        
        curr_chap, latest_chap = func(lines[1])
        current.append(curr_chap)
        latest.append(latest_chap)

In [424]:
cur.execute("CREATE TABLE Manga(title, url, current, latest)")

In [425]:
insert = list(zip(titles, urls, current, latest))


('Magic Emperor',
 'https://chapmanganato.to/manga-gr983826',
 'Chapter 1',
 'Chapter 559')

In [426]:
cur.executemany("INSERT INTO Manga VALUES(?, ?, ?, ?)", insert)

In [430]:
cur.close()
con.close()